# core

> Generate a definition card for a youtube video

In [1]:
#| default_exp core

In [3]:
#| hide
from nbdev.showdoc import *

In [30]:
#| export

import re
from youtube_transcript_api import YouTubeTranscriptApi
from fastcore.xtras import store_attr, basic_repr
from fastcore.script import *
from claudette import *
import os
from bs4 import BeautifulSoup
from bs4.formatter import HTMLFormatter
from fastcore.script import *


In [5]:
#| export

def extract_video_id(url_or_id):
    if len(url_or_id) == 11: return url_or_id
    
    patterns = [
        r'(?:youtube\.com\/watch\?v=|youtu\.be\/|youtube\.com\/embed\/|youtube\.com\/v\/|youtube\.com\/watch\?.*v=)([^&\n?#]+)',
        r'youtube\.com\/shorts\/([^&\n?#]+)'
    ]
    
    for pattern in patterns:
        match = re.search(pattern, url_or_id)
        if match: return match.group(1)
    
    return url_or_id

def get_transcript(url_or_id, languages=['en']):
    video_id = extract_video_id(url_or_id)
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=languages)
    return ' '.join(item['text'] for item in transcript_list)


In [6]:
# txt_1 = get_transcript('https://youtu.be/zC44sB9EWcw?si=DqZBwKwdAU_j9CB3')
# txt_1


In [7]:
#| export

os.environ['ANTHROPIC_LOG'] = 'debug'
model = models[1]; model

'claude-3-7-sonnet-20250219'

In [8]:
#| export

class Definition:
    "Information about a president of the United States"
    def __init__(self, 
                guest_name:str, # The full name of the guest     
                guest_position:str, # The position of the guest
                definition:str, # The definition of what an AI agent is according to the guest
        ):
        store_attr()

    __repr__ = basic_repr('guest_name, guest_position, definition')

In [9]:
#| export

def get_definition(txt):
    sp="""You will receive a podcast transcript about AI agents. Summarize the definition according to the guest.
    Keep it short, 2 or 3 sentences max.
    <example>
    A piece of software you can delegate tasks to, empowering and augmenting human agency rather than acting as a separate entity. These agents use LLMs and APIs to act in the world based on requests and can be easily replicated. They are fundamentally tools extending human capabilities under human direction.
    </example>
    """
    cli = Client(model)
    pr = f"{sp}<transcript>{txt}</transcript>"
    r = cli.structured(pr, Definition)
    return r[0]

In [10]:
#| export

def parse_markdown_links(text):
    """Convert markdown-style links [text](url) to HTML <a> tags"""
    import re
    pattern = r'\[(.*?)\]\((.*?)\)'
    
    def replace_func(match):
        text, url = match.groups()
        return f'<a href="{url}" target="_blank">{text}</a>'
    
    return re.sub(pattern, replace_func, text)

def generate_definition_card(youtube_link, guest_name='', guest_position=''):
    txt = get_transcript(youtube_link)
    d = get_definition(txt)
    # if name and position are not provided, use the extracted ones
    if guest_name == '': guest_name = d.guest_name
    if guest_position == '': guest_position = d.guest_position
    
    # Parse markdown-style links in guest name and position
    html_guest_name = parse_markdown_links(guest_name)
    html_guest_position = parse_markdown_links(guest_position)
    
    card_html = f'''
<div class="definition-card">
    <div class="meta">
        <div>
            <h2>{html_guest_name}</h2>
            <span class="role">{html_guest_position}</span>
        </div>
        <a href="{youtube_link}" class="watch-link" target="_blank">
            <span>Watch on</span>
            <svg class="youtube-icon" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" fill="currentColor">
                <path d="M23.498 6.186a3.016 3.016 0 0 0-2.122-2.136C19.505 3.545 12 3.545 12 3.545s-7.505 0-9.377.505A3.017 3.017 0 0 0 .502 6.186C0 8.07 0 12 0 12s0 3.93.502 5.814a3.016 3.016 0 0 0 2.122 2.136c1.871.505 9.376.505 9.376.505s7.505 0 9.377-.505a3.015 3.015 0 0 0 2.122-2.136C24 15.93 24 12 24 12s0-3.93-.502-5.814zM9.545 15.568V8.432L15.818 12l-6.273 3.568z"/>
            </svg>
        </a>
    </div>
    <p>{d.definition}</p>
</div>
'''
    return card_html.strip()


In [11]:
# card = generate_definition_card('https://www.youtube.com/watch?v=kTmUW86-HSE&t=913s')

In [12]:
#| export

def insert_card_into_html(card_html, html_file_path='what-is-an-ai-agent.html'):
    """
    Insert a definition card into the definitions section of the HTML file.
    
    Args:
        card_html (str): The HTML for the card to insert
        html_file_path (str): Path to the HTML file
        
    Returns:
        bool: True if successful, False otherwise
    """
    try:
        # Read the existing HTML file
        with open(html_file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # Find the definitions section
        definitions_marker = '<div class="definitions">'
        if definitions_marker not in content:
            print(f"Error: Could not find '{definitions_marker}' in the HTML file.")
            return False
        
        # Insert the new card after the opening div
        insert_position = content.find(definitions_marker) + len(definitions_marker)
        
        # Add some indentation and newlines for clean formatting
        formatted_card = "\n" + card_html
        # Insert the card
        new_content = content[:insert_position] + formatted_card + content[insert_position:]
        
        # Write the updated content back to the file
        with open(html_file_path, 'w', encoding='utf-8') as f:
            f.write(new_content)
        
        print(f"Successfully inserted card into {html_file_path}")
        return True
    
    except Exception as e:
        print(f"Error inserting card: {str(e)}")
        return False

In [13]:
# insert_card_into_html(card)

In [14]:
#| export
def add_new_definition(youtube_link, guest_name, guest_position, html_file_path='what-is-an-ai-agent.html'):
    card = generate_definition_card(youtube_link, guest_name, guest_position)
    insert_card_into_html(card, html_file_path)


In [15]:
add_new_definition('https://youtu.be/-aBD8i2_mGM?si=5HxahyxtEM_Fl7K1', 'Thorsten Ball', 'Softwate Engineer @ Sourcegraph of Aomni', html_file_path='../what-is-an-ai-agent.html')

Successfully inserted card into ../what-is-an-ai-agent.html


In [17]:
#| export
def extract_definitions(html_file='what-is-an-ai-agent.html'):
    """Extract all definition texts from existing cards"""
    soup = BeautifulSoup(open(html_file).read(), 'html.parser')
    cards = soup.select('.definition-card')
    
    return [{
        'name': card.select_one('h2').text.strip(),
        'role': card.select_one('.role').text.strip(),
        'definition': card.select_one('p').text.strip()
    } for card in cards if not (card.select_one('h2') and 'Latest Consensus' in card.select_one('h2').text)]


In [18]:
# defs = extract_definitions('../what-is-an-ai-agent.html')
# defs

In [19]:
#| export 
def get_consensus(definitions):
    sp="""You will receive a list of definitions of AI agents. Summarize the consensus according to the definitions.
    Keep it short, 2 or 3 sentences max.
    <example>
    A piece of software you can delegate tasks to, empowering and augmenting human agency rather than acting as a separate entity. These agents use LLMs and APIs to act in the world based on requests and can be easily replicated. They are fundamentally tools extending human capabilities under human direction.
    </example>
    """
    c = Chat(model)
    # cli = Client(model)
    pr = f"{sp}<definitions>{definitions}</definitions>"
    r = c(pr)
    return r

In [20]:
# r = get_consensus(defs)
# r

In [28]:
#| export
def create_consensus_html(consensus_text):
    """Create HTML for consensus card"""
    return f'''
<div class="definition-card consensus-card">
    <div class="meta">
        <div>
            <h2>Latest Consensus</h2>
            <span class="role">Synthesized from all experts</span>
        </div>
    </div>
    <p>{consensus_text}</p>
</div>'''.strip()

def insert_consensus(html_file='what-is-an-ai-agent.html'):
    """Insert consensus card at top of definitions section"""
    # Get content and definitions
    content = open(html_file).read()
    soup = BeautifulSoup(content, 'html.parser')
    
    # Get definitions and generate consensus
    definitions = extract_definitions(html_file)
    consensus = get_consensus(definitions)
    card_html = create_consensus_html(contents(consensus))
    
    # Remove existing consensus card if present
    existing_consensus = soup.select_one('.consensus-card')
    if existing_consensus:
        existing_consensus.decompose()
    
    # Get definitions container
    definitions_div = soup.select_one('.definitions')
    
    # Create new consensus card element
    new_card = BeautifulSoup(card_html, 'html.parser')
    
    # Insert at the beginning of definitions div
    definitions_div.insert(0, new_card)
    
    # Format the HTML with proper indentation
    formatter = HTMLFormatter(indent=4)
    pretty_html = soup.prettify(formatter=formatter)
    
    # Save file
    with open(html_file, 'w') as f:
        f.write(pretty_html)
    
    return consensus

In [ ]:
# insert_consensus('../what-is-an-ai-agent.html')

AI agents are software systems that handle complex, long-tail use cases by adapting to obstacles through non-deterministic interactions, moving beyond traditional interfaces to more natural ones like chat or voice. They represent a new paradigm of software that empowers users by delegating tasks through multiple prompts and models working together, ultimately extending human capabilities while remaining under human direction.

<details>

- id: `msg_01Nug2UML4TAZSAZ1Qbp6a7z`
- content: `[{'citations': None, 'text': 'AI agents are software systems that handle complex, long-tail use cases by adapting to obstacles through non-deterministic interactions, moving beyond traditional interfaces to more natural ones like chat or voice. They represent a new paradigm of software that empowers users by delegating tasks through multiple prompts and models working together, ultimately extending human capabilities while remaining under human direction.', 'type': 'text'}]`
- model: `claude-3-7-sonnet-20250219`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 533, 'output_tokens': 77}`

</details>

In [24]:
#| export

@call_parse
def add_new_guest(youtube_link: str, # The youtube link of the guest
                  guest_name: str, # The name of the guest (can contain markdown links [text](url))
                  guest_position: str, # The position of the guest (can contain markdown links [text](url))
                  html_file_path: str = 'what-is-an-ai-agent.html'):
    card = generate_definition_card(youtube_link, guest_name, guest_position)
    insert_card_into_html(card, html_file_path)
    insert_consensus(html_file_path)



In [ ]:
add_new_guest('https://youtu.be/-40h0O6olus?si=w_7XiJdC3T2P5aru', 'Arjun Chintapalli & Bhavani Kalisetty', 'rtrvr.ai', html_file_path='../what-is-an-ai-agent.html')

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()